In [7]:
import cv2
import numpy as np

from PIL import Image
from tqdm import tqdm
from cyvlfeat.sift.dsift import dsift
from cyvlfeat.kmeans import kmeans
from scipy.spatial.distance import cdist

In [2]:
CAT = ['Kitchen', 'Store', 'Bedroom', 'LivingRoom', 'Office',
       'Industrial', 'Suburb', 'InsideCity', 'TallBuilding', 'Street',
       'Highway', 'OpenCountry', 'Coast', 'Mountain', 'Forest']

CAT2ID = {v: k for k, v in enumerate(CAT)}

In [10]:
path = '/home/r12942159/NTU_CV/Hw2/hw2_data/p1_data/train/Bedroom/image_0001.jpg'

In [18]:
img = cv2.imread(path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
tiny_2D = cv2.resize(img, (16, 16), interpolation=cv2.INTER_AREA)
tiny_1D = tiny_2D.flatten().astype(np.float32)

mean = np.mean(tiny_1D)
tiny_1D -= mean

norm = np.linalg.norm(tiny_1D)
tiny_1D /= (norm + 1e-8)

In [ ]:
dsift_step = 4
vocab_size=100
batch_size = 243
features = []

img = cv2.imread(path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
frames, descriptors = dsift(
    img, step=[dsift_step, dsift_step], fast=True)

for i in range(descriptors.shape[0]):
    features.append(descriptors[i])

num_batches = len(features) // batch_size + (1 if len(features) % batch_size != 0 else 0)
batches_features = np.array_split(features, num_batches)

# Perform k-means clustering on each subset of features
vocab = np.empty((0, batches_features[0].shape[1]), np.float32)
for batch in batches_features:
    batch_vocab, _ = kmeans(batch.astype(np.float32), vocab_size)[:2]
    vocab = np.vstack((vocab, batch_vocab))

In [ ]:
img_feats = []
dsift_step = 4
num_samples = 1000

img = np.asarray(Image.open(path), dtype='float32')
frames, descriptors = dsift(
    img, step=[dsift_step, dsift_step], fast=True
)

if len(descriptors) > num_samples:
    idx = np.random.choice(
        len(descriptors), size=num_samples, replace=False)
    descriptors = descriptors[idx]

dist = cdist(vocab, descriptors, metric='euclidean')
idx = np.argmin(dist, axis=0)
hist, _ = np.histogram(idx, bins=len(vocab))
hist_norm = [float(i)/sum(hist) for i in hist]
